In [1]:
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
!tar xvzf ldcc-20140209.tar.gz

Streaming output truncated to the last 5000 lines.
text/smax/smax-6833065.txt
text/smax/smax-6833494.txt
text/smax/smax-6833579.txt
text/smax/smax-6834422.txt
text/smax/smax-6834562.txt
text/smax/smax-6834765.txt
text/smax/smax-6834990.txt
text/smax/smax-6835318.txt
text/smax/smax-6838218.txt
text/smax/smax-6838260.txt
text/smax/smax-6838332.txt
text/smax/smax-6838453.txt
text/smax/smax-6838514.txt
text/smax/smax-6838627.txt
text/smax/smax-6838702.txt
text/smax/smax-6842164.txt
text/smax/smax-6842219.txt
text/smax/smax-6842269.txt
text/smax/smax-6842402.txt
text/smax/smax-6842629.txt
text/smax/smax-6842882.txt
text/smax/smax-6843206.txt
text/smax/smax-6845003.txt
text/smax/smax-6847081.txt
text/smax/smax-6849018.txt
text/smax/smax-6850114.txt
text/smax/smax-6850391.txt
text/smax/smax-6850789.txt
text/smax/smax-6851590.txt
text/smax/smax-6851919.txt
text/smax/smax-6853063.txt
text/smax/smax-6853346.txt
text/smax/smax-6853751.txt
text/smax/smax-6854198.txt
text/smax/smax-6855347.txt
text

In [2]:
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.996.5
#pip install mecab-python3
!pip install pyyaml==5.4.1
!pip install unidic-lite nlplot japanize-matplotlib transformers fugashi ipadic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
  swig3.0
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl lib

In [3]:
#データセットの読み込み
import os
import re
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()
import nlplot
import warnings
warnings.simplefilter('ignore')    
import torch
import transformers
from transformers import BertJapaneseTokenizer
import logging
logging.getLogger("transformers.tokenization_utils_base").setLevel(logging.ERROR) # tokenize時の警告を抑制
def load_dataset():
    """データセットの読み込み"""
    paths = []
    for dirpath, dirnames, filenames in os.walk('./text'):
        for file in filenames:
            if re.match(r'.+[0-9].txt',  file):
                #print("{0}".format(file))
                paths.append(os.path.join(dirpath, file))                
    data = {
        'path': [],
        'URL': [],
        'date': [],
        'title': [],
        'text': [],
    }
    for path in paths:
        with open(path, 'r') as f:
            url = f.readline().strip('¥n')
            date = f.readline().strip('¥n')
            title = f.readline().strip('¥n')
            text = f.read()
            data['path'].append(path)
            data['URL'].append(url)
            data['date'].append(date)
            data['title'].append(title)
            data['text'].append(text)
    return pd.DataFrame(data)
#データセットのパス、記事のURL、日付、記事のタイトル、本文を格納した DataFrame を返す
df = load_dataset()

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [96]:
class BertExtractor:
    """文書特徴抽出用クラス"""
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu' #GPUが使用可能ならGPUを使用
        self.model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking' #使用する学習済みモデル名
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(self.model_name) #使用するBERTトークナイザー
        self.bert_model = transformers.BertModel.from_pretrained(self.model_name) #学習済みモデル呼び出し
        self.bert_model = self.bert_model.to(self.device)
        self.max_len = 128 #使用する入力文書の長さ。最大512まで
    def extract(self, sentence):
        """文書特徴ベクトルを抽出する"""
        # 文書のトークナイズ
        inp = self.tokenizer.encode(sentence)
        len_inp = len(inp)
        # 入力トークン数の調整
        if len_inp >= self.max_len:
            inputs = inp[:self.max_len]
        else:
            inputs=inp
        #else:
        #    inputs = inp + [0] * (self.max_len - len_inp) #padding
        # モデルへ文書を入力し特徴ベクトルを取り出す
        inputs_tensor = torch.tensor([inputs], dtype=torch.long).to(self.device)        
        seq_out = self.bert_model(inputs_tensor)[0]
        if torch.cuda.is_available():    
            return seq_out[0][0].cpu().detach().numpy() # 0番目は [CLS] token, 768 dim の文章特徴量
        else:
            return seq_out[0][0].detach().numpy()

In [97]:
def cos_sim_matrix(matrix):
    """文書間のコサイン類似度を計算し、類似度行列を返す"""
    d = matrix @ matrix.T 
    norm = (matrix * matrix).sum(axis=1, keepdims=True) ** .5
    return d / norm / norm.T
bex = BertExtractor()
df['text_feature'] = df['text'].progress_apply(lambda x: bex.extract(x)) # 文書の特徴ベクトル化
sim=cos_sim_matrix(np.stack(df.text_feature)) # 類似度行列

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/7367 [00:00<?, ?it/s]

In [103]:
#類似文書の検索
#ランダムに記事を一つ選び、その記事と類似度が高い記事を100件取り出す
def search(n=100):
    doc = df.sample(1)
    doc_idx = doc.index[0]
    sim_index = sim[doc_idx].argsort()[::-1]
    rec_df = df.iloc[sim_index][:n]
    rec_df['similarity'] = np.sort(sim[doc_idx])[::-1][:n]
    return rec_df[['title', 'text', 'similarity']]
df2 = search()
df2.head(10)

,title,text,similarity
2191,美しき処刑人、「アンダーワールド」シリーズ第4弾の敵は“人類”\n,2003年に第1弾『アンダーワールド』が公開され、2006年に第2弾『アンダーワールド：エ...,1.000000
2510,ケイト・ベッキンセール主演『アンダーワールド 覚醒』が全米初登場1位\n,美しきケイト・ベッキンセールがヴァンパイア処刑人を演じる人気シリーズ「アンダーワールド」。...,0.915352
2003,ロサンゼルスでトワイライト現象！映画のために5日間テントで泊まり込み\n,"原作本の売り上げが世界で1億冊、映画シリーズの世界累計興行収入が18億ドル（1,450億円...",0.906698
2499,野郎ども、バグにツイートをくれてやれ！『スターシップ・トゥルーパーズ：インベイジョン』キャン...,ＳＦ小説の巨匠ロバート・Ａ・ハインラインの傑作『宇宙の戦士』を映画化し、巨大昆虫と戦う兵士...,0.902322
2406,寝室で一体何が起こったのか、『パラノーマル』最新作が衝撃映像を限定公開\n,世界的大ヒットを記憶したホラー映画『パラノーマル・アクティビティ』のシリーズ最新作『パラノ...,0.897689
2126,『スパイダーマン』新ヒロインのエマ・ストーン「素敵なキスシーンはあります！」\n,2002年に公開された『スパイダーマン』に始まり、2004年公開の『スパイダーマン2』、2...,0.896739
2034,死ぬまで戦え！ 『スターシップ・トゥルーパーズ』昆虫と人類の壮絶な戦いが再び\n,SF小説の巨匠ロバート・A・ハインラインの傑作「宇宙の戦士」の映画化にして、クエンティン・...,0.894942
2163,先行上映が決定、ついに“アベンジャーズ現象”が日本上陸\n,『アバター』『タイタニック』に迫る大ヒットを記録し、全世界歴代興行収入第3位にランクインし...,0.889102
1958,映画『ベルセルク』ポスター＆予告編が解禁、ED曲はAIが担当\n,全世界累計3000万部突破、連載20年以上に及ぶ三浦建太郎原作の「ベルセルク」。その長大な...,0.888917
1844,『ベルセルク』ファン必見、キャラクターのレアアイテムが登場\n,映画『ベルセルク』の世界をiPhoneへ！ 全世界累計3300万部を超えるファンタジー超大...,0.887118


In [99]:
df['title_feature'] = df['title'].progress_apply(lambda x: bex.extract(x)) # 文書の特徴ベクトル化

  0%|          | 0/7367 [00:00<?, ?it/s]

In [ ]:
#検索結果の可視化
def tokenize(text,):
    tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
    wakati_ids = tokenizer.encode(text, return_tensors='pt')
    tokens = tokenizer.convert_ids_to_tokens(wakati_ids[0].tolist())
    return tokens[1:-1] #[CLS], [SEP]トークンを除く
df2['tokenized_text'] = df2['text'].progress_apply(tokenize)

npt = nlplot.NLPlot(df2, target_col='tokenized_text')
# top_nで頻出上位単語, min_freqで頻出下位単語を指定できる
stopwords = npt.get_stopword(top_n=20, min_freq=0)
# bi-gram表示
npt.bar_ngram(
    title='bi-gram',
    xaxis_label='word_count',
    yaxis_label='word',
    ngram=2,
    top_n=50,
    stopwords=stopwords,
)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:00<00:00, 1919.63it/s]


In [ ]:
#wordcloud表示
npt.wordcloud(
    max_words=100,
    max_font_size=100,
    colormap='tab20_r',
    stopwords=stopwords,
)

In [ ]:
#よりテキスト長の短いタイトルで特徴ベクトル化を行う
def cos_sim_matrix(matrix):
    """文書間のコサイン類似度を計算し、類似度行列を返す"""
    d = matrix @ matrix.T 
    norm = (matrix * matrix).sum(axis=1, keepdims=True) ** .5
    return d / norm / norm.T
df['text_feature'] = df['title'].progress_apply(lambda x: bex.extract(x)) # 文書の特徴ベクトル化
sim=cos_sim_matrix(np.stack(df.text_feature)) # 類似度行列

In [104]:
df[0:10]["title"]

0    【Sports Watch／競馬】 心ない野次に騎手が激怒“「死ね」だけは許せない”\n
1                【Sports Watch】大相撲再開も、取組に変化が・・・\n
2              ペナルティ・ワッキー、W杯で遠藤が決めたフリーキックの裏話明かす\n
3                         インテル弱体化で、長友の価値も下がる?  \n
4          【Sports Watch】桑田の講義に、魔裟斗とラモスが真っ向から反論\n
5        【Sports Watch】北島康介、試合前のヘッドホン姿に、ある理由が——\n
6                結束空しく一敗地に塗れた男子柔道、五輪代表は不本意な選出に？\n
7                    金本の永久欠番内定に野球ファン「安い永久欠番ですね」\n
8                  なでしこジャパンに“意外な共通点” 18人中17人が……\n
9                 スター性抜群、次代を担う“超美少女スケーター”今井遥に注目\n
Name: title, dtype: object

In [132]:
words=["北島康介","フリーキック","W杯","ワールドカップ","競馬","なでしこジャパン"]
for word in words:
  a=bex.extract(word)
  #b=df[3:4]["title_feature"].values[0]
  b=bex.extract("【Sports Watch】北島康介、試合前のヘッドホン姿に、ある理由が——\n")
  cos_sim = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)) 
  print("{0}: {1}".format(word,cos_sim))


北島康介: 0.745169997215271
フリーキック: 0.7298997044563293
W杯: 0.6246761679649353
ワールドカップ: 0.6372961401939392
競馬: 0.7434582114219666
なでしこジャパン: 0.6981901526451111


In [144]:
'''def cos_sim_matrix(matrix):
    """文書間のコサイン類似度を計算し、類似度行列を返す"""
    d = matrix @ matrix.T 
    norm = (matrix * matrix).sum(axis=1, keepdims=True) ** .5
    return d / norm / norm.T
sim=cos_sim_matrix(np.stack(df.text_feature)) # 類似度行列
'''
word="スポーツウォッチ"
dic={}
for i in range(100):
  a=bex.extract(word)
  b=df[i:i+1]["title_feature"].values[0]
  cos_sim = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)) 
  dic[cos_sim]=df[i:i+1]["title"]
order= sorted(dic.items(), reverse=True)
order

[(0.7353471, 44    アーセナルに「忘れられた？」　パク・チュヨンが記載漏れ \n
  Name: title, dtype: object),
 (0.7318297, 51    【Sports Watch】宮里藍、今シーズン好調の理由を語る\n
  Name: title, dtype: object),
 (0.73160297, 67    【Sports Watch】大健闘・村上佳菜子、「ビックリしてフワフワ」\n
  Name: title, dtype: object),
 (0.72174877, 71    【Sports Watch】“左のおかわり君”坂田「去年は全く歯が立たなかった」\n
  Name: title, dtype: object),
 (0.71912205, 33    【Sports Watch】松井大輔＆ローサの結婚。芸能レポーターも「全く知りませんでした」\n
  Name: title, dtype: object),
 (0.7163245, 92    【Sports Watch】上田桃子「この度、結婚致します」\n
  Name: title, dtype: object),
 (0.7158906, 40    【Sports Watch】茨城GG・片岡安祐美、監督一年目は「毎日泣きたい」\n
  Name: title, dtype: object),
 (0.71387994, 36    【Sports Watch】喜びの声続々、奥寺氏「カメルーンはバラバラ」\n
  Name: title, dtype: object),
 (0.7103946, 46    【Sports Watch】プロレスのオールスター戦開催も、猪木は呼ばれず激怒\n
  Name: title, dtype: object),
 (0.705399, 23    【Sports Watch】イチローの米球宴落選に、“過去の活躍とか関係ない”\n
  Name: title, dtype: object),
 (0.7049447, 62    「勘違いしすぎ」片岡安祐美の発言に批判の声 \n
  Name: title, dtype: object),
 (0.7042128, 94    【Sports W

In [116]:
class BertExtractor:
    """文書特徴抽出用クラス"""
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu' #GPUが使用可能ならGPUを使用
        self.model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking' #使用する学習済みモデル名
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(self.model_name) #使用するBERTトークナイザー
        self.bert_model = transformers.BertModel.from_pretrained(self.model_name) #学習済みモデル呼び出し
        self.bert_model = self.bert_model.to(self.device)
        self.max_len = 128 #使用する入力文書の長さ。最大512まで
    def extract(self, sentence):
        """文書特徴ベクトルを抽出する"""
        # 文書のトークナイズ
        inp = self.tokenizer.encode(sentence)
        len_inp = len(inp)
        # 入力トークン数の調整
        if len_inp >= self.max_len:
            inputs = inp[:self.max_len]
        #else:
        #    inputs = inp + [0] * (self.max_len - len_inp) #padding
        # モデルへ文書を入力し特徴ベクトルを取り出す
        inputs_tensor = torch.tensor([inp], dtype=torch.long).to(self.device)        
        seq_out = self.bert_model(inputs_tensor)[0]
        if torch.cuda.is_available():    
            return seq_out[0][0].cpu().detach().numpy() # 0番目は [CLS] token, 768 dim の文章特徴量
        else:
            return seq_out[0][0].detach().numpy()

In [128]:
bex = BertExtractor()
words=["オープンカラーシャツ","シャツ","上着","ズボン","下着","モンハン","夏目漱石","二・二六事件"]
for word in words:
  a=bex.extract(word)
  b=bex.extract("オーバーサイズオープンカラーシャツ(Blue) Mサイズ 2980円")
  cos_sim = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
  print("{0}: {1}".format(word,cos_sim))

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


オープンカラーシャツ: 0.8328887224197388
シャツ: 0.7214111685752869
上着: 0.7125694155693054
ズボン: 0.6771097779273987
下着: 0.6751765608787537
モンハン: 0.5038548111915588
夏目漱石: 0.5692082047462463
二・二六事件: 0.5837154984474182


In [ ]:
#類似文書の検索
#ランダムに記事を一つ選び、その記事と類似度が高い記事を100件取り出す
def search(n=100):
    doc = df.sample(1)
    doc_idx = doc.index[0]
    sim_index = sim[doc_idx].argsort()[::-1]
    rec_df = df.iloc[sim_index][:n]
    rec_df['similarity'] = np.sort(sim[doc_idx])[::-1][:n]
    return rec_df[['title', 'text', 'similarity']]
df2 = search()
